In [1]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from numpy import asarray
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
#from kmodes.kmodes import KModes

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore") 

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!ls "/content/drive/My Drive/TFM_Velarte"

'car-causas-agrupadas(v2).ipynb'
'car-Cleaning_and_Clustering(v6).ipynb'
'car_Cleaning_and_Clustering(v7).ipynb'
'car-Cleaning_and_Clustering(v7).ipynb'
 Catboost.ipynb
 catboost.pickle
 Cleaning_and_Clustering.ipynb
'Cleaning_and_Clustering(v3) (1).ipynb'
'Cleaning_and_Clustering(v3).ipynb'
'Cleaning_and_Clustering(v4) (1).ipynb'
'Cleaning_and_Clustering(v4).ipynb'
'Copia de causas210918.ipynb'
'Copia de Cleaning_and_Clustering.ipynb'
 Dashboard
 DF_CLEAN_VELARTE.csv
 DF_SEGUN_VELARTE2.csv
 DF_SEGUN_VELARTE3csv.csv
 DF_SEGUN_VELARTE.csv
 ETL.ipynb
 img
 Info
'Informe Producción y Mermas.xlsx'
 Meetings
 NEW.ipynb
 OLD
'Presentación y Memoria'
 Report_TFM.gdoc
 resultados_articulos_a_predecir.csv
 resultados_predicciones_produccion.csv
 VELARTE_MERMA_CAUSAS.csv
 VELARTE_MERMA_CAUSAS.gsheet
 VELARTE_PROD_MERMA.csv
 VELARTE_PROD_MERMA.gsheet


In [4]:
path= '/content/drive/My Drive/TFM_Velarte/VELARTE_PROD_MERMA.csv'
delimiter = ','
df = pd.read_csv(path, header='infer', delimiter=delimiter)

In [5]:
df.head()

,Orden,Fecha,Articulo_Orden,Cantidad orden,Cantidad Merma,Cod_ Turno Trabajo,Location Code
0,L16/06321,2016-01-03,9510701,180.0,0.0,NaN,L1
1,L16/00001,2016-01-03,9030101,170.0,0.0,L1-3,L1
2,L16/00009,2016-01-04,9510207,20.0,13.9,L2-3,L2
3,L16/00028,2016-01-04,9510205,80.0,0.0,L2-3,L2
4,L16/00027,2016-01-04,9510107,180.0,0.0,L2-3,L2


In [6]:
df.shape

(42189, 7)

In [7]:
del df['Orden']

In [8]:
df.head()

,Fecha,Articulo_Orden,Cantidad orden,Cantidad Merma,Cod_ Turno Trabajo,Location Code
0,2016-01-03,9510701,180.0,0.0,NaN,L1
1,2016-01-03,9030101,170.0,0.0,L1-3,L1
2,2016-01-04,9510207,20.0,13.9,L2-3,L2
3,2016-01-04,9510205,80.0,0.0,L2-3,L2
4,2016-01-04,9510107,180.0,0.0,L2-3,L2


In [9]:
df['Codigo_rechazo']=100

In [10]:
df.head()

,Fecha,Articulo_Orden,Cantidad orden,Cantidad Merma,Cod_ Turno Trabajo,Location Code,Codigo_rechazo
0,2016-01-03,9510701,180.0,0.0,NaN,L1,100
1,2016-01-03,9030101,170.0,0.0,L1-3,L1,100
2,2016-01-04,9510207,20.0,13.9,L2-3,L2,100
3,2016-01-04,9510205,80.0,0.0,L2-3,L2,100
4,2016-01-04,9510107,180.0,0.0,L2-3,L2,100


In [11]:
#Reemplazamos según los insights
df['Codigo_rechazo'] = np.where((df['Articulo_Orden']==9050542), 35, df['Codigo_rechazo'])
df['Codigo_rechazo'] = np.where((df['Articulo_Orden']==9050541), 31, df['Codigo_rechazo'])
df['Codigo_rechazo'] = np.where((df['Articulo_Orden']==9050272), 32, df['Codigo_rechazo'])

df['Codigo_rechazo'] = np.where((df['Articulo_Orden']==9010470), 71, df['Codigo_rechazo'])
df['Codigo_rechazo'] = np.where((df['Articulo_Orden']==9010527), 71, df['Codigo_rechazo'])
df['Codigo_rechazo'] = np.where((df['Articulo_Orden']==9010610), 71, df['Codigo_rechazo'])
df['Codigo_rechazo'] = np.where((df['Articulo_Orden']==9010414), 71, df['Codigo_rechazo'])

df['Codigo_rechazo'] = np.where((df['Articulo_Orden']==9020101), 81, df['Codigo_rechazo'])
df['Codigo_rechazo'] = np.where((df['Cantidad Merma']<8), 372, df['Codigo_rechazo'])

In [12]:
df.head()

,Fecha,Articulo_Orden,Cantidad orden,Cantidad Merma,Cod_ Turno Trabajo,Location Code,Codigo_rechazo
0,2016-01-03,9510701,180.0,0.0,NaN,L1,372
1,2016-01-03,9030101,170.0,0.0,L1-3,L1,372
2,2016-01-04,9510207,20.0,13.9,L2-3,L2,100
3,2016-01-04,9510205,80.0,0.0,L2-3,L2,372
4,2016-01-04,9510107,180.0,0.0,L2-3,L2,372


In [13]:
#Creamos la columna causa
df['Causa']=100

In [14]:
#Ahora rellenamos la columna causa en funcion del codigo rechazo
df['Causa'] = np.where((df['Codigo_rechazo']==35), 2, df['Causa']) #Envasado
df['Causa'] = np.where((df['Codigo_rechazo']==31), 2, df['Causa'])
df['Causa'] = np.where((df['Codigo_rechazo']==32), 2, df['Causa'])
df['Causa'] = np.where((df['Codigo_rechazo']==71), 1, df['Causa']) #Film
df['Causa'] = np.where((df['Codigo_rechazo']==81), 4, df['Causa']) #Horno
df['Causa'] = np.where((df['Codigo_rechazo']==372), 3, df['Causa']) #Elaboración

In [15]:
df['Tipo']=100

#Reemplazamos las que teníamos por insights y existen en el excel
df['Tipo'] = np.where((df['Articulo_Orden']==9050542), 2, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9050541), 2, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9050272), 2, df['Tipo'])

df['Tipo'] = np.where((df['Articulo_Orden']==9010470), 2, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9010527), 1, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9010610), 1, df['Tipo'])

#Las top 20 que tienen más merma
df['Tipo'] = np.where((df['Articulo_Orden']==9010610), 1, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9010496), 2, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9020110), 1, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9050270), 2, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9510325), 2, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9010323), 1, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9010467), 1, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9510415), 2, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9010466), 1, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9050574), 2, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9010494), 2, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9010611), 1, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9020111), 1, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9010475), 2, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9010321), 1, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9010105), 1, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9010486), 2, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9010499), 1, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9050547), 2, df['Tipo'])
df['Tipo'] = np.where((df['Articulo_Orden']==9010321), 1, df['Tipo'])

In [16]:
df.head(50)


,Fecha,Articulo_Orden,Cantidad orden,Cantidad Merma,Cod_ Turno Trabajo,Location Code,Codigo_rechazo,Causa,Tipo
0,2016-01-03,9510701,180.0,0.00,NaN,L1,372,3,100
1,2016-01-03,9030101,170.0,0.00,L1-3,L1,372,3,100
2,2016-01-04,9510207,20.0,13.90,L2-3,L2,100,100,100
3,2016-01-04,9510205,80.0,0.00,L2-3,L2,372,3,100
4,2016-01-04,9510107,180.0,0.00,L2-3,L2,372,3,100
5,2016-01-04,9510101,288.0,0.00,L1-3,L1,372,3,100
6,2016-01-04,9500302,91.0,1.10,L2-2,L2,372,3,100
7,2016-01-04,9050324,80.0,0.00,L2-3,L2,372,3,100
8,2016-01-04,9050323,137.0,0.00,L2-3,L2,372,3,100
9,2016-01-04,9050320,720.0,0.00,L2-2,L2,372,3,100


In [ ]:
df[df['Codigo_rechazo']!=100].shape[0]

30926

In [ ]:
df['Fecha'] = pd.to_datetime(df['Fecha'])

In [ ]:
df['Weekday'] = df['Fecha'].dt.dayofweek

In [ ]:
df['Day'] = pd.DatetimeIndex(df['Fecha']).day
df['Month'] = pd.DatetimeIndex(df['Fecha']).month
df['Year'] = pd.DatetimeIndex(df['Fecha']).year

In [ ]:
del df['Fecha']

In [ ]:
df.dtypes

Articulo_Orden          int64
Cantidad orden        float64
Cantidad Merma        float64
Cod_ Turno Trabajo     object
Location Code          object
Codigo_rechazo          int64
Causa                   int64
Tipo                    int64
Weekday                 int64
Day                     int64
Month                   int64
Year                    int64
dtype: object

In [ ]:
df.isna().sum()

Articulo_Orden           0
Cantidad orden           0
Cantidad Merma           0
Cod_ Turno Trabajo    3769
Location Code           98
Codigo_rechazo           0
Causa                    0
Tipo                     0
Weekday                  0
Day                      0
Month                    0
Year                     0
dtype: int64

In [ ]:
df['Cod_ Turno Trabajo'].unique()

array([nan, 'L1-3', 'L2-3', 'L2-2', 'L2-1', 'L1-1', 'L1-2', 'M',
       'AJUSTE  L1', 'MASA  L2', 'L2-FD-3', 'L2-FD-1', 'L2-FD-2',
       'L1-FD-2', 'L1-FD-1', 'L2-TE-3', 'L2-TE-1', 'L1-TE-3', 'L1-FD-3',
       'L1-TE-1', 'L1-TE-2'], dtype=object)

In [ ]:
index_na = df[df['Cod_ Turno Trabajo'].isna()].index
# drop these row indexes
# from dataFrame
df.drop(index_na, inplace = True)

In [ ]:
index_na2 = df[df['Location Code'].isna()].index
# drop these row indexes
# from dataFrame
df.drop(index_na2, inplace = True)

In [ ]:
index_LR = df[df['Location Code']=='LR'].index
# drop these row indexes
# from dataFrame
df.drop(index_LR, inplace = True)

In [ ]:
df['Location Code']= df['Location Code'].replace(['L1'], value='1')
df['Location Code']= df['Location Code'].replace(['L2'], value='2')
df['Location Code']= df['Location Code'].replace(['L3'], value='3')
df['Cod_ Turno Trabajo']= df['Cod_ Turno Trabajo'].replace(['L1-1'], value='1')
df['Cod_ Turno Trabajo']= df['Cod_ Turno Trabajo'].replace(['L1-2'], value='2')
df['Cod_ Turno Trabajo']= df['Cod_ Turno Trabajo'].replace(['L1-3'], value='3')
df['Cod_ Turno Trabajo']= df['Cod_ Turno Trabajo'].replace(['L2-1'], value='1')
df['Cod_ Turno Trabajo']= df['Cod_ Turno Trabajo'].replace(['L2-2'], value='2')
df['Cod_ Turno Trabajo']= df['Cod_ Turno Trabajo'].replace(['L2-3'], value='3')
df['Cod_ Turno Trabajo']= df['Cod_ Turno Trabajo'].replace(['L3-1'], value='1')
df['Cod_ Turno Trabajo']= df['Cod_ Turno Trabajo'].replace(['L3-2'], value='2')
df['Cod_ Turno Trabajo']= df['Cod_ Turno Trabajo'].replace(['L3-3'], value='3')
df['Cod_ Turno Trabajo']= df['Cod_ Turno Trabajo'].replace(['AJUSTE  L1', 'AJUSTE  L2', 'MASA  L2', 'M', 'L2-FD-3',
       'L2-FD-2', 'L2-FD-1', 'L1-FD-1', 'L1-TE-1', 'L1-FD-3', 'L1-TE-3',
       'L1-FD-2', 'L2-TE-1', 'L2-TE-3', 'T', 'L1-TE-2'], value='OTRO')

In [ ]:
index_m = df[df['Cod_ Turno Trabajo']=='OTRO'].index
# drop these row indexes
# from dataFrame
df.drop(index_m, inplace = True)

In [ ]:
index_names100 = df[df['Codigo_rechazo']==100].index
# drop these row indexes
# from dataFrame
df.drop(index_names100, inplace = True)

In [ ]:
df['Articulo_Orden'] = df['Articulo_Orden'].astype(int)
df['Cod_ Turno Trabajo'] = df['Cod_ Turno Trabajo'].astype(int)
df['Location Code'] = df['Location Code'].astype(int)
df['Day'] = df['Day'].astype(int)
df['Month'] = df['Month'].astype(int)
df['Year'] = df['Year'].astype(int)

In [ ]:
df.tail(50)

,Articulo_Orden,Cantidad orden,Cantidad Merma,Cod_ Turno Trabajo,Location Code,Codigo_rechazo,Causa,Tipo,Weekday,Day,Month,Year
29417,9500320,3.0,0.00,2,1,372,3,100,2,26,6,2019
29418,9500300,192.0,0.00,2,1,372,3,100,2,26,6,2019
29419,9500299,453.0,0.00,3,1,372,3,100,2,26,6,2019
29420,9500299,336.0,0.00,2,1,372,3,100,2,26,6,2019
29423,9050573,52.0,0.00,3,2,372,3,100,2,26,6,2019
29425,9050543,37.0,0.00,1,2,372,3,100,2,26,6,2019
29427,9050541,782.0,78.10,2,2,31,2,2,2,26,6,2019
29428,9050541,640.0,30.50,1,2,31,2,2,2,26,6,2019
29429,9050541,186.0,13.30,3,2,31,2,2,2,26,6,2019
29430,9050348,240.0,0.00,3,1,372,3,100,2,26,6,2019


In [ ]:
df.shape

(26676, 12)

In [ ]:
df = df[df['Year']!=2020]
df = df[(df['Year']!=2021) & (df['Month']>=1) & (df['Month']<7)]

In [ ]:
df.shape

(11601, 12)

In [ ]:
df.dtypes

Articulo_Orden          int64
Cantidad orden        float64
Cantidad Merma        float64
Cod_ Turno Trabajo      int64
Location Code           int64
Codigo_rechazo          int64
Causa                   int64
Tipo                    int64
Weekday                 int64
Day                     int64
Month                   int64
Year                    int64
dtype: object

In [ ]:
df.to_csv('DF_CLEAN_VELARTE.csv', index=False)